# 00_Introduction_to_bioImages

***

This notebook serves as an introduction to bioimages. We will take you through some basic concepts demonstrating how to load images and plot images in a Jupyter notebook, how to change the contrast of your images, how images are actually just arrays of numbers and how to deal with image dimensionality. 

In [ ]:
from glob import glob
from skimage import io
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt
import stackview
import numpy as np

## <font color='seagreen'> 0: Loading and inspecting Images </font>

In the next code block, we will assign the paths to our images to the variable 'DIC_paths'. Then we will use the function io.imread from scikit-image (a python image processing library) to load our images into memory. 

In [ ]:
path=fr'.\data\00_Intro_to_BioImages\*.tif'
DIC_paths=glob(path)
DIC_paths

In [ ]:
DICs= [io.imread(DIC_path) for DIC_path in DIC_paths]

Okay, now that we have loaded our images into memory we can have a look at their attributes (number of images, data-type,  their shapes, minimal pixel intensity value, maximal intensity value).

In [ ]:
print(len(DICs))

In [ ]:
DICs[2]

In [ ]:
DICs[2].shape

In [ ]:
DICs[2].min()

In [ ]:
DICs[2].max()

**Q0.0: Can an image be described as an array of numbers?**

type here

**Q0.1: What are the shapes of each image?**

type here

**Q0.2: Our images are 16-bit images. What is the minimum and maximal possible intensity for a single pixel in our images?**

type here

**Q0.3: Are their differences between the minimum and maximum pixel intensity of each image?**

type here

***

## <font color='seagreen'> 1: Playing with Contrast </font> 

Let us plot the images that we have loaded. We have loaded three images corresponding to the fungi: Candida albicans, Arbuscular Mycorrhizal Fungi, and Saccharomyces cerevisiae. The images are all brightfield microscopy images. However, for two of the images, a DIC-filter (https://en.wikipedia.org/wiki/Differential_interference_contrast_microscopy) was used during acquisition.  

In [ ]:
fig,ax = plt.subplots(ncols=3, figsize=(20,20))

for i in range(0,3):
    ax[i].imshow(DICs[i],cmap='Greys_r', vmin=np.percentile(DICs[i],10),vmax=np.percentile(DICs[i],90))
    ax[i].set_title(Path(DIC_paths[i]).stem)

**Q1.0 Do all the images have the same contrast and brightness?**

type here

We can count the number of pixels with a certain intensity and display these within a histogram. If we plot these histograms for all our images, we can see how the pixel intensity distributions differ per image. 

In [ ]:
fig,ax = plt.subplots(ncols=3, figsize=(15,3))

for i in range(0,3):
    ax[i].hist(DICs[i].ravel(), 255)
    ax[i].set_title(Path(DIC_paths[i]).stem)

    #set x lims and labels
    ax[i].set_xlim([0,2**16])
    ax[i].set_xlabel('Fluorescence intensity (A.U.)')
    
    #set y lims and labels
    ax[i].set_ylim([0,800000])
    if i==0:
        ax[i].set_ylabel('Counts')

**Q1.1 Do the intensity distribution correspond to the perceived brightness in the plotted images below (yes/no). Why do you think this is the case?**

type here

We can determine to which minimal and maximal pixel intensity our colourmap is scaled. Change these values and inspect what their effect is on the contrast and brightness of the image below.

In [ ]:
#change minimal value for colourmap mapping
min_pixel=800

#change maximal value for colourmap mapping
max_pixel=10000

plt.figure()

plt.imshow(DICs[1], cmap='Greys_r', vmin=min_pixel,vmax=max_pixel)
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label('fluorescence (A.U)')

Lets change the colour map used.

In [ ]:
#change minimal value for colourmap mapping
min_pixel=800

#change maximal value for colourmap mapping
max_pixel=10000

plt.figure()

plt.imshow(DICs[1], cmap='viridis', vmin=min_pixel,vmax=max_pixel)
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label('Title (Unit)')

**Q1.2 The image has now different colours, has this changed anything to the raw image data?**

type here.

Do the same but for all images at the same time. 

In [ ]:
fig,ax = plt.subplots(ncols=3, nrows=2, figsize=(20,10))

# every pixel below this value is displayed as black
min_pixel=800

# every pixel above this value is displayed at maximum brightness
max_pixel=40000

# add a colourbar to your plot
norm = mpl.colors.Normalize(vmin=min_pixel, vmax=max_pixel)

#How does this number change the mapping or binning of your colourbar?
N=100
#change Greys_r to viridis and see what happens :)
cmap = plt.get_cmap('Greys_r',N) 

#Create a mapped colourbar figure.
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm) 
sm.set_array([]) 
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(sm, cax=cbar_ax)

#plot images and histograms
for i in range(0,3):
    #display images
    ax[0,i].imshow(DICs[i],cmap=cmap, vmin=min_pixel,vmax=max_pixel)
    ax[0,i].set_title(Path(DIC_paths[i]).stem)

    #display histograms
    ax[1,i].hist(DICs[i].ravel(), 255)
    ax[1,i].set_title(Path(DIC_paths[i]).stem)

    #set x lims and labels for every histogram
    ax[1,i].set_xlim([0,2**16])
    ax[1,i].set_xlabel('Fluorescence intensity (A.U.)')
    
    #set y lims and labels every histogram
    ax[1,i].set_ylim([0,800000])
    if i==0:
        ax[i,1].set_ylabel('Counts')

    #add vertical bars for vmin and vmax to each histogram
    ax[1,i].axvline(x=min_pixel, c='red')
    ax[1,i].axvline(x=max_pixel, c='red')

**Q1.3 What effect has the variable N on the mapping of your colourmap?**

type here

**Q1.4 After setting vmin and vmax, can the brightness of each image now be better compared? (explain why or why not)**

type here

***

## <font color='seagreen'> 2: Dealing with Dimensionality </font> 

So far we have only dealt with 2D images with a x, and y dimension. However, images can be multidimensional as well. For example, a timelapse videos of growing cells or a multilayered image. Here, we will open 2 multidimensional images (image stacks), and give you some idea of how to deal with these larger formats. Particularly, we will focus on how to index and display certain parts of these images.

In [ ]:
path=r'.\data\00_Intro_to_BioImages\multiD_images\*.tif'
image_stack_paths=glob(path)
image_stack_paths

In [ ]:
stack1, stack2 = io.imread(image_stack_paths[0]), io.imread(image_stack_paths[1])

In [ ]:
stack1.shape

In [ ]:
stack2.shape

**Q2.0 Inspect the shape of each image, how many dimensions do they have?**

type here

In [ ]:
stackview.slice(stack1, zoom_factor=0.3)

**Q2.1 Let's have a closer look at stack1. What dimension are displayed in this image?** 

type here

**Exercise**

Display the 40th image in this timelapse video of growing $C.albicans$ cells.

In [ ]:
stack1.shape

In [ ]:
img = stack1[] # index inside the square brackets

plt.imshow(img, cmap='Greys_r')

stack2 is not a timelapse video but a multilayered and multichannel fluorescent imagestack instead. Its image dimensions (3, 41, 2304, 2304) correspond to the dimensions: (channel, layer, x, y). Different fluorescent channels can be used to measure the fluorescence of dyes and molecules within the cell. In this case, channel 0 corresponds to cell autofluorescence, whilst in channel a specific mRNA molecule is stained, and in channel 2 the cell nucleus is stained. The code block below can be used to scroll through the z-layers of a single channel.

In [ ]:
stackview.slice(stack2[2,:,...], zoom_factor=0.3)

Here, we will perform a maximimun intensity over one of the image dimension in stack2. Such that only the maximum pixel intensities across that dimension are kept.

In [ ]:
zproject_stack2= np.amax(stack2[:,22:26,...],axis=1)
zproject_stack2.shape

**Q2.1 How does a z-maxprojection change the shape of your imagestack?**

type here

In [ ]:
stackview.switch(
    {"empty":  zproject_stack2[0,...],
     "mRNA":  zproject_stack2[1,...],
     "nucleus":  zproject_stack2[2,...]
    },
    colormap=["pure_magenta", "pure_green", "pure_blue"],
    toggleable=True, 
    zoom_factor=0.3
)

**2.2 Inspect the remaining dimension of your zprojection. Which dimension was lost?**

type here

***

## <font color='seagreen'> 3: Programming excersize </font> 

**3.1 Inspect the image below. Use indexing to crop and display a single cell**

In [ ]:
#change this code
io.imshow(DICs[-1])

**3.1 Change the brightness, contrast, and the colormap of the image**

In [ ]:
#change this code
io.imshow(DICs[-1])

***

You have reached the end of this notebook, have you understood the main concepts in this introductions about image analysis?

- Images are just big multidimensional arrays of numbers.

- Contrast and brightness are display settings that can be changed. Be careful about using the same settings when comparing images. 

- In microscopy colour often correspond to colourmaps and not to actual colours. Therefore, image's colour can be changed without changing the underlying raw data.

See you in the next notebook.